In [1]:
import math
import torch
import tenseal as ts
import numpy as np
from skimage.util.shape import view_as_windows

In [2]:
def almost_equal(vec1, vec2, m_pow_ten):
    if len(vec1) != len(vec2):
        return False

    upper_bound = pow(10, -m_pow_ten)
    for v1, v2 in zip(vec1, vec2):
        if abs(v1 - v2) > upper_bound:
            return False
    return True

### Torch 2d convolution

In [3]:
def torch_conv_2d(x, kernel, stride):
    return torch.nn.functional.conv2d(
        input=x, weight=kernel, stride=stride, padding=0, dilation=1
    )

### Generate input

In [4]:
# input image dimension n * n
x_size = 4
# kernel dimension n * n
k_size = 2
# stride
stride = 1

# generated incremeneted values: 1, 2, ..., n^2
x = np.arange(1, x_size ** 2 + 1).reshape(x_size, x_size)
kernel = np.arange(1, k_size ** 2 + 1).reshape(k_size, k_size)

# generated random values
# x = np.random.randn(x_size, x_size)
# kernel = np.random.randn(k_size, k_size)

print("input", x.shape)
print(x)
print("kernel", kernel.shape)
print(kernel)

input (4, 4)
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
kernel (2, 2)
[[1 2]
 [3 4]]


### TenSEAL context

In [5]:
# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60]
)
# set the scale
context.global_scale = pow(2, 40)
# generated galois keys in order to do rotation on ciphertext vectors
context.generate_galois_keys()

For each convolution layer, a communication between the client and server is required. The server send the ciphertext (encrypted vector) to the client which is the input of the next convolution layer, in order to decrypt it and apply im2col (Image Block to Column) on the that input.

After that the client encode and encrypt the input matrix in a vertical scan (column-major) and send it back to the server.

In [6]:
%%time 

x_enc, windows_nb = ts.im2col_encoding(context, x, kernel.shape[0], kernel.shape[1], stride)

print("windows number: ", windows_nb)
print("ckksvector size: ", x_enc.size())

y_enc = x_enc.conv2d_im2col(kernel.tolist(), windows_nb)

print(y_enc.size())
y_plain = y_enc.decrypt()

print("y_enc")
print(y_plain)

windows number:  9
ckksvector size:  36
9
y_enc
[44.00000566594027, 54.00000698409561, 64.00000865907336, 84.0000112607995, 94.0000125997811, 104.00001396028475, 124.00001662478137, 134.00001797146558, 144.00001931406726]
CPU times: user 25.7 ms, sys: 0 ns, total: 25.7 ms
Wall time: 24.6 ms


### Compare the result to torch conv2d

In [7]:
y_torch = torch_conv_2d(
    torch.from_numpy(x.astype("float32")).unsqueeze(0).unsqueeze(0),
    torch.from_numpy(kernel.astype("float32")).unsqueeze(0).unsqueeze(0),
    stride
)
y_torch = y_torch.flatten().numpy()
print("y_toch")
print(y_torch)

assert almost_equal(y_plain, y_torch, 0)

y_toch
[ 44.  54.  64.  84.  94. 104. 124. 134. 144.]
